In [1]:
import openstudio

# pip install fuzzywuzzy[speedup]
from fuzzywuzzy import process

I'm comparing the IDD references for objects referencing curves between E+ and OS IDDs.
The fuzzywuzzy python module is used to match the OS field name to the EP IDD one (because we may be outdated or deviated on purpose, like removing numbers when making extensible or removing 'Name' at end)

See https://github.com/NREL/OpenStudio/issues/4289

In [10]:
ep_idd_path = "../resources/energyplus/ProposedEnergy+.idd"
ep_idd = openstudio.IddFile.load(openstudio.toPath(ep_idd_path)).get()

In [3]:
os_idd_path = "../resources/model/OpenStudio.idd"
os_idd = openstudio.IddFile.load(openstudio.toPath(os_idd_path)).get()

In [4]:
curve_objects = [x for x in os_idd.objects() if ("Curve:" in x.name()) or ("Table:") in x.name()]
set([x for curve in curve_objects for x in curve.references() if not "UniqueNames" in x])

{'AllCurves',
 'BivariateFunctions',
 'MultivariateFunctions',
 'QuadvariateFunctions',
 'QuintvariateFunctions',
 'TrivariateFunctions',
 'UnivariateFunctions'}

In [12]:
CURVE_REFS = {
    "AllCurves",
    "BivariateFunctions",
    "MultivariateFunctions",
    "QuadvariateFunctions",
    "QuintvariateFunctions",
    "TrivariateFunctions",
    "UnivariateFunctions",
}

In [13]:
for obj in os_idd.objects():
    obj_name = obj.name()
    if any(ref in CURVE_REFS for ref in obj.objectLists()):
        # print(obj_name)

        ep_obj_name = obj_name.replace("OS:", "")
        o_ep_obj = ep_idd.getObject(ep_obj_name)
        if not o_ep_obj.is_initialized():
            print(f"failed to locate in EP IDD for {obj_name}")
            continue

        ep_obj = o_ep_obj.get()
        ep_obj_field_names = [ep_obj.getField(i).get().name() for i in range(ep_obj.numFields())]

        for idx in obj.objectListFields():
            field = obj.getField(idx).get()
            field_name = props = field.name()

            props = field.properties()
            refs = set([x for x in props.objectLists if not "AllCurve" in x])
            if not any(ref in CURVE_REFS for ref in refs):
                continue

            # print(f" * {field_name}: {refs}")

            ep_field_name, score = process.extractOne(field_name, ep_obj_field_names)
            if score < 80:
                print(f"Bad score for {obj_name} - {ep_field_name}: {score}")

            else:
                # print(f"{score} - {field_name} - {ep_field_name}")
                pass
            ep_field_idx = ep_obj_field_names.index(ep_field_name)
            ep_field = ep_obj.getField(ep_field_idx).get()
            ep_refs = set([x for x in ep_field.properties().objectLists])
            extra_os = refs - ep_refs

            if extra_os:
                print(f"WRONG extra OS {obj_name} - {field_name}: {refs=}, {ep_refs=}, {ep_field_name=}")

            extra_ep = ep_refs - refs
            if extra_ep:
                print(f"WRONG extra EP {obj_name} - {field_name}: {refs=}, {ep_refs=}")

failed to locate in EP IDD for OS:ElectricEquipment:ITE:AirCooled:Definition
failed to locate in EP IDD for OS:Coil:Cooling:FourPipeBeam
failed to locate in EP IDD for OS:Coil:Heating:FourPipeBeam
failed to locate in EP IDD for OS:Coil:Cooling:DX:MultiSpeed:StageData
failed to locate in EP IDD for OS:Coil:Heating:Gas
failed to locate in EP IDD for OS:Coil:Heating:DX:MultiSpeed:StageData
failed to locate in EP IDD for OS:Coil:Heating:DX:VariableSpeed:SpeedData
failed to locate in EP IDD for OS:Coil:Cooling:DX:VariableSpeed:SpeedData
failed to locate in EP IDD for OS:Coil:Heating:WaterToAirHeatPump:VariableSpeedEquationFit:SpeedData
failed to locate in EP IDD for OS:Coil:Cooling:WaterToAirHeatPump:VariableSpeedEquationFit:SpeedData
failed to locate in EP IDD for OS:Coil:WaterHeating:AirToWaterHeatPump
failed to locate in EP IDD for OS:Coil:WaterHeating:AirToWaterHeatPump:VariableSpeed:SpeedData
failed to locate in EP IDD for OS:Generator:MicroTurbine:HeatRecovery
failed to locate in EP I

These above I manually checked